In [1]:
#1/12/24 Creates and saves the models for:
#ChemoPy | RF | Ames_Mutagenicity

import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename and 'Mutagen' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 50):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    sets += 1

For dataset Ames_Mutagenicity_Chemopy.csv
for fold 1 test set mcc of 0.683847970598678 valid set mcc of 0.6377752589459977
for fold 2 test set mcc of 0.6308899719970773 valid set mcc of 0.6464629492789153
for fold 3 test set mcc of 0.6421868463725344 valid set mcc of 0.6877582882060621
for fold 4 test set mcc of 0.6859767635798173 valid set mcc of 0.6308359017106373
for fold 5 test set mcc of 0.6490713996780497 valid set mcc of 0.5867106854580441

validation metrics of:
positives in data 840
negatives in data 657
net accuracy = 0.8396793587174348
mcc = 0.6737881829840559


In [8]:
import joblib
loops = 0
for item in model_list:
    temp = loops
    loops = str(loops)
    name = loops + '.pkl'
    path = '/Users/james/Documents/Honours/Addenum/Model storage/ChemoPy/RF/Ames_Mutagenicity/' + name
    
    joblib.dump(item, path) 
    loops = temp + 1

In [7]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/Chemopy/resultsdata/chemopy_rf.csv')

In [8]:
datasets[6]

,Toxicity_Value,Weight,nH,nHal,nHet,nHA,nF,nCl,nBr,nI,...,VSAEstate0,VSAEstate1,VSAEstate2,VSAEstate3,VSAEstate4,VSAEstate5,VSAEstate6,VSAEstate7,VSAEstate8,VSAEstate9
0,0,148.150,4,0,5,10,0,0,0,0,...,0.547454,13.559583,0.000000,1.539352,0.000000,0.000000,3.115231,0.000000,0.000000,4.905046
1,0,392.198,16,6,16,28,6,0,0,0,...,79.365588,13.706120,6.623437,3.618548,-4.964313,0.501815,-4.874915,-0.226280,0.000000,0.000000
2,0,249.040,7,2,9,16,0,2,0,0,...,0.000000,3.800942,8.238624,11.965103,0.176914,5.134145,0.000000,0.000000,0.000000,11.906493
3,0,388.264,21,2,24,30,2,0,0,0,...,26.426309,14.554939,20.051051,2.034770,-1.100900,17.734904,-0.034407,0.000000,0.000000,0.000000
4,0,446.378,28,0,28,34,0,0,0,0,...,6.779758,16.071759,20.445076,2.038770,1.029534,19.265172,3.943649,4.134694,0.000000,1.458255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,1,362.287,24,0,23,29,0,0,0,0,...,2.126948,18.888857,11.077501,3.931990,0.196565,19.636096,5.457594,2.351115,0.000000,0.000000
1543,0,439.753,27,1,25,33,0,1,0,0,...,2.116340,26.333584,15.836239,3.421882,-0.736528,12.776628,4.340082,5.500209,0.000000,6.356008
1544,0,482.331,35,4,29,37,4,0,0,0,...,51.952916,19.354981,0.000000,4.824498,-1.218842,13.241029,-0.196909,4.458993,0.000000,0.000000
1545,0,403.311,29,1,26,32,1,0,0,0,...,15.162470,21.379596,0.000000,5.203739,0.029218,13.726178,9.481959,3.850174,0.000000,0.000000
